# Session 4: Data Structuring 3

*Nicklas Johansen*

## Agenda

In this session, we will continue our work with `pandas` and data structuring:

- Missings and Duplicated Data
- Combining Data Sets
- Split-Apply-Combine
- Rehaping Data

## Recap 

- Series & DataFrames
- Boolean Data
- Numeric Operations and Methods
- String Operations
- Categorical Data
- Time Series Data

*But I'm nowhere near an expert in data structuring 1 and 2 :( ...*

**Don't worry**
- You have lot's of time to catch up!
- Solutions will help you along the way
- We teach you all the tool; but you only need some of them to in your exam

## Questions past week
- Dictionaries in multiple layers
- Naming index keys
- Method chaining

### Dictionaries in Multiple Layers

In the exercises, you encountered a problem with navigating in dictionaries with multiple layers. Why are these interesting?
- It clearly illustrates how dictionaries are (not) related in structure
    - DataFrames have a matrix-like (2D) structure
    - Dictionaries have a non-sequential, 1D structure
    - In order to represent a df-like object with dictionaries, we must combine them in multiple layers
- Data that come in the form of JSON will often have this structure
    - Potentially in many more layers

In [11]:
my_dict1 = {0: 432, 1: 654, 2: 776 , 3: 234, 4: 765}
my_dict2 = {0: 131, 1: 432, 2: 876 , 3: 432, 4: 174}
my_dict3 = {0: 222, 1: 654, 2: 23 , 3: 862, 4: 175}

my_d = {"first": my_dict1, "second": my_dict2, "third": my_dict3}

In [8]:
my_d

{'first': {0: 432, 1: 654, 2: 776, 3: 234, 4: 765},
 'second': {0: 131, 1: 432, 2: 876, 3: 432, 4: 174},
 'third': {0: 222, 1: 654, 2: 23, 3: 862, 4: 175}}

In [9]:
my_d['first']

{0: 432, 1: 654, 2: 776, 3: 234, 4: 765}

In [10]:
my_d['first'][2]

776

### Naming Columns and Rows

We can name columns. And we can name rows. Both are easy:

In [14]:
my_df = pd.DataFrame(np.arange(9).reshape(3,3))
my_df

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8


In [15]:
cols = []
rows = []
[rows.append(f'row{i + 1}') for i in range(3)] # you can also write ('row{a}'.format(a=i+1))
[cols.append('column'+str(i+1)) for i in range(3)]
             
my_df.columns, my_df.index = cols, rows
my_df

,column1,column2,column3
row1,0,1,2
row2,3,4,5
row3,6,7,8


### Method Chaining

Make multiple operations in one line! Be careful that your code remains readible...

In [16]:
my_df_new = my_df\
                .rename(columns = {'column1': 'COL100', 'column2': 'COL200', 'column3': 'COL300'})\
                .assign(my_sum = lambda my_df: my_df['COL100']*100+my_df['COL200']*10+my_df['COL300'])\
                .sort_values(by='my_sum', ascending = False)\
                .reset_index(drop=True)
my_df_new

,COL100,COL200,COL300,my_sum
0,6,7,8,678
1,3,4,5,345
2,0,1,2,12


What did that lambda function just do? Essentially, the idea is that it is a function that takes $\textbf{x}$ as input:

In [18]:
func = lambda x1, x2, x3: x1+x2+x3
func(1,2,3)

6

In [19]:
# Loading packages

import numpy as np
import pandas as pd
import seaborn as sns

tips = sns.load_dataset('tips')
titanic = sns.load_dataset('titanic')

#import matplotlib.pyplot as plt
#import requests

# Today: More Structuring :-)

In practice, you are rarely handed some data that are just ready for analysis. In particular, it may suffer from:

- Presence of missings and/or duplicated values:
    - missing: Should we ignore? delete? replace (impute)? collect new data?
        - Depends on the context!
    - duplicated: Are they present? Should they be removed?
        - Depends on the context!

- Your data comes in separate portions that must be combined:
    - Vertical merge? Horizontal merge?
    - Inner or outer merge? On which keys?

- Your data may have the different shape:
    - What is a long format and a wide format?
    - How to shift between the two?

- You may need some aggregate pieces of information on different subgroups?
    - How to group-specific means, medians, variances, etc?

# Missing Data

## Representation of Missing Data
*How can we think about missing data?*

Missing data, i.e. empty observations:
- In Python: `None`
- In pandas: numpy's 'not a number' abbreviated with `NaN` or simply `nan`

## Missing Data in Pandas
*What does a DataFrame with missing data look like*

In [20]:
nan_data = [[0,np.nan,1],\
            [2,3,None],\
            [4,5,6]]

nan_df = pd.DataFrame(nan_data, columns = ['A', 'B', 'C'])
nan_df

,A,B,C
0,0,NaN,1.0
1,2,3.0,NaN
2,4,5.0,6.0


In [21]:
nan_df.isnull()

,A,B,C
0,False,True,False
1,False,False,True
2,False,False,False


## Creating Missings (1:2)

*Would we ever want to create missings?*

In the ideal world, no. In practice, yes:
- Sometimes, missings are coded to a numerical value.
- These can be very influential in your data analysis if not uncoverred.

Example:

In [22]:
nan_data2 = [[0,999999,1],\
            [2,3,999999],\
            [4,5,6]]

pd.DataFrame(nan_data2, columns = ['A', 'B', 'C'])

,A,B,C
0,0,999999,1
1,2,3,999999
2,4,5,6


## Creating Missings (2:2)

The solutions is simple:

In [23]:
pd.DataFrame(nan_data2, columns = ['A', 'B', 'C']).replace(999999,np.NaN)

,A,B,C
0,0,NaN,1.0
1,2,3.0,NaN
2,4,5.0,6.0


## Imputing Missings
*Are there ways to replace missings with values with pandas?*

As mentioned earlier, pandas does handle missings directly. See table 7-2 in PDA. Examples:

In [24]:
nan_df.fillna(method='bfill') # can also use 'ffill'

,A,B,C
0,0,3.0,1.0
1,2,3.0,6.0
2,4,5.0,6.0


## Dropping Missings
*Are there also ways to simply drop missings with pandas?*

Yes, this is fortunately easy. Use the `.dropna()` feature in pandas:
- Remember to drop along the right axis.

In [28]:
nan_df.dropna(axis=0) # , thresh=2

,A,B,C
2,4,5.0,6.0


In [29]:
nan_df.dropna(axis=1)

,A
0,0
1,2
2,4


*Note:* For practical purposes, you risk dropping too many observations here!

# Duplicated Data

## Duplicates: A Definition
*What does it mean that there are duplicates in the data?*

Slightly tricky to define:
- More than one entry where there should in fact be only one.
- If for a certain set of variables, a combination is repeated.

In practice, it requires some understanding of your data:
- Two observations are identical: Is this a duplicate, or is it truly because 'occurance happened twice'?
- In income registries from DK: One individual should not show up more than once per year in the data.

## Handling Duplicates (1:2)
*In practice, what could a duplicate look like?*

Let's revisit our categorical education data:

In [33]:
edu_list = ['BSc Political Science', 'Secondary School'] + ['High School']*2
edu_cats = ['Secondary School', 'High School', 'BSc Political Science']
str_ser = pd.Series(edu_list*10**5)

str_ser.head(6)

0    BSc Political Science
1         Secondary School
2              High School
3              High School
4    BSc Political Science
5         Secondary School
dtype: object

Now, let's see what is duplicated:

In [35]:
str_ser.duplicated().head(6) # can also specify: ", keep='last'"

0    False
1    False
2    False
3     True
4     True
5     True
dtype: bool

## Handling Duplicates (2:2)
*How do we drop duplicates?*

Simply use the `.drop_duplicates()` method.

In [17]:
str_ser.drop_duplicates()

0    BSc Political Science
1         Secondary School
2              High School
dtype: object

## Multiple Variables (1:2)
*How do we drop duplicates when we have multiple variables (e.g. persons and years)?*

Simulate an additional binary variable for purposes of illustration:

In [36]:
edu_df = pd.DataFrame({'edu': edu_list*10**5, 'num': np.round(np.random.rand(4*10**5),0)})
edu_df.head(10)

,edu,num
0,BSc Political Science,1.0
1,Secondary School,0.0
2,High School,1.0
3,High School,1.0
4,BSc Political Science,1.0
5,Secondary School,0.0
6,High School,0.0
7,High School,1.0
8,BSc Political Science,1.0
9,Secondary School,1.0


## Multiple Variables (2:2)
Now, specify the combination of columns for which we require unique values when dropping:

In [37]:
edu_df.drop_duplicates(['edu', 'num'], keep='first')

,edu,num
0,BSc Political Science,1.0
1,Secondary School,0.0
2,High School,1.0
6,High School,0.0
9,Secondary School,1.0
24,BSc Political Science,0.0


# Joining DataFrames

## Joining DataFrames

Until now, we've worked with one DataFrame at a time:
- with the exception of `concat` in Assignment 0

We will now learn to put them together.

For the following, we use the multi column display from Jake van der Plaas [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html).

In [41]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


## Some DataFrames
Let's make some data to play with. Here, use `dict` approach:

In [42]:
left = pd.DataFrame({'key': ['A', 'B', 'C', 'D'], 'value_left': range(4)})    
right = pd.DataFrame({'key': ['C', 'D', 'E', 'F'], 'value_right': range(4,8)})
display('left', 'right')

,key,value_left
0,A,0
1,B,1
2,C,2
3,D,3
,key,value_right
0,C,4
1,D,5
2,E,6
3,F,7


## Merging Data
The forthcoming figures all follow this convention:

-  <font color="blue">blue</font>: rows in merge output
-  <font color="red">red</font>: rows excluded from output (i.e., removed)
-  <font color="green">green</font>: missing values replaced with NaNs 

We use `merge` which is pandas function and a method for dataframes.

## Inner Merge (default)
This merge only uses only *shared* keys

In [43]:
inner_merge = pd.merge(left, right, on='key', how='inner')
inner_merge

,key,value_left,value_right
0,C,2,4
1,D,3,5


<center><img src='https://i.stack.imgur.com/YvuOa.png' alt="Drawing" style="width: 400px;"/></center>

## Left Merge
This merge uses only *left* keys

In [44]:
left_merge = pd.merge(left, right, on='key', how='left')
left_merge

,key,value_left,value_right
0,A,0,NaN
1,B,1,NaN
2,C,2,4.0
3,D,3,5.0


<center><img src='https://i.stack.imgur.com/BECid.png' alt="Drawing" style="width: 400px;"/></center>

## Right Merge
This merge uses only *right* keys (pretty redundant...)

In [45]:
right_merge = pd.merge(left, right, on='key', how='right')
right_merge

,key,value_left,value_right
0,C,2.0,4
1,D,3.0,5
2,E,NaN,6
3,F,NaN,7


<center><img src='https://i.stack.imgur.com/8w1US.png' alt="Drawing" style="width: 400px;"/></center>

## Outer Merge
This merge uses *all* keys

In [46]:
outer_merge = pd.merge(left, right, on='key', how='outer')
outer_merge

,key,value_left,value_right
0,A,0.0,NaN
1,B,1.0,NaN
2,C,2.0,4.0
3,D,3.0,5.0
4,E,NaN,6.0
5,F,NaN,7.0


<center><img src='https://i.stack.imgur.com/euLoe.png' alt="Drawing" style="width: 400px;"/></center>

## Overview of Merge Types

<center><img src='https://www.dofactory.com/Images/sql/sql-joins.png' alt="Drawing" style="width: 450px;"/></center>

More merge type exists, see [this post](https://stackoverflow.com/questions/53645882/pandas-merging-101) for details.

## Joining DataFrames

We can also join by keys in the index. This is possible with `join` or `concat`.

Additional properties of `concat`:
- works vertically and horizontally.
- works with multiple DataFrames at once;

Requirement: overlapping index keys or column names.

In [47]:
df0 = left.set_index('key')
df1 = right.set_index('key')

In [48]:
df0

,value_left
key,
A,0
B,1
C,2
D,3


## Horizontal Join 

Works like `merge` where _key_ is now the index! 

In [49]:
inner_data = df0.join(df1, how='inner')
inner_data

,value_left,value_right
key,,
C,2,4
D,3,5


## Horizontal Join, Axis = 1

`concat` on axis=1 acts similar to a outer join.

In [50]:
dfs = [df0, df1]
pd.concat(dfs, axis=1, sort=False)

,value_left,value_right
key,,
A,0.0,NaN
B,1.0,NaN
C,2.0,4.0
D,3.0,5.0
E,NaN,6.0
F,NaN,7.0


## Vertical Join, Axis = 0

On the other hand, `concat` on axis=0 stacks the dataframes on top of each other!

In [51]:
pd.concat([df0, df1], join='outer', axis=0, sort=False)

,value_left,value_right
key,,
A,0.0,NaN
B,1.0,NaN
C,2.0,NaN
D,3.0,NaN
C,NaN,4.0
D,NaN,5.0
E,NaN,6.0
F,NaN,7.0


## Vertical and Horizontal

An overview of `concat` operations (left: horizontal, right: vertical)

<center><img src='https://i.stack.imgur.com/1rb1R.jpg' alt="Drawing" style="width: 450px;"/></center>

## Example of Using Concat (1:3)

Let's load some stock data...

In [53]:
import yfinance as yf
import matplotlib.pyplot as plt # For plotting

stocks = ['aapl', 'goog', 'msft', 'amzn', 'fb', 'tsla']

def load_stock(s):
    return yf.download(s, data_source='yahoo', start='2000-01-01')['Adj Close']

stock_dfs = {s:load_stock(s) for s in stocks} # dictionary of all stock price
stock_df = pd.concat(stock_dfs, axis=1) # horizontal join

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,aapl,goog,msft,amzn,fb,tsla
Date,,,,,,
1999-12-31,0.788183,NaN,36.920834,76.125000,NaN,NaN
2000-01-03,0.858137,NaN,36.861530,89.375000,NaN,NaN
2000-01-04,0.785788,NaN,35.616329,81.937500,NaN,NaN
2000-01-05,0.797286,NaN,35.991867,69.750000,NaN,NaN
2000-01-06,0.728291,NaN,34.786213,65.562500,NaN,NaN
...,...,...,...,...,...,...
2021-09-22,145.850006,2818.770020,298.579987,3380.050049,343.209991,751.940002
2021-09-23,146.830002,2836.530029,299.559998,3416.000000,345.959991,753.640015
2021-09-24,146.919998,2852.659912,299.350006,3425.520020,352.959991,774.390015


In [59]:
stock_dfs

{'aapl': Date
 1999-12-31      0.788183
 2000-01-03      0.858137
 2000-01-04      0.785788
 2000-01-05      0.797286
 2000-01-06      0.728291
                  ...    
 2021-09-22    145.850006
 2021-09-23    146.830002
 2021-09-24    146.919998
 2021-09-27    145.369995
 2021-09-28    141.910004
 Name: Adj Close, Length: 5471, dtype: float64,
 'goog': Date
 2004-08-19      49.982655
 2004-08-20      53.952770
 2004-08-23      54.495735
 2004-08-24      52.239197
 2004-08-25      52.802086
                  ...     
 2021-09-22    2818.770020
 2021-09-23    2836.530029
 2021-09-24    2852.659912
 2021-09-27    2830.020020
 2021-09-28    2723.679932
 Name: Adj Close, Length: 4308, dtype: float64,
 'msft': Date
 1999-12-31     36.920834
 2000-01-03     36.861530
 2000-01-04     35.616329
 2000-01-05     35.991867
 2000-01-06     34.786213
                  ...    
 2021-09-22    298.579987
 2021-09-23    299.559998
 2021-09-24    299.350006
 2021-09-27    294.170013
 2021-09-28    283.

## Example of Using Concat (2:3)

What do the data look like?

In [55]:
stock_df.tail(5)

,aapl,goog,msft,amzn,fb,tsla
Date,,,,,,
2021-09-22,145.850006,2818.770020,298.579987,3380.050049,343.209991,751.940002
2021-09-23,146.830002,2836.530029,299.559998,3416.000000,345.959991,753.640015
2021-09-24,146.919998,2852.659912,299.350006,3425.520020,352.959991,774.390015
2021-09-27,145.369995,2830.020020,294.170013,3405.800049,353.579987,791.359985
2021-09-28,141.910004,2723.679932,283.519989,3315.959961,340.649994,777.559998


## Example of Using Concat (3:3)

Now, let's try and visualize it (more about this in next session)!

In [56]:
ax = stock_df.plot(logy=True, figsize=(10,3))
ax.legend(["Apple", "Google","Microsoft", "Amazon","Facebook", "Tesla"], loc='best', ncol=2)

# Split-Apply-Combine

## A Definition
*What is the split-apply-combine framework?*

A procedure to...
1. **split** a DataFrame into subsets of data,
2. **apply** certain functions (sorting, mean, other custom stuff), and
3. **combine** it back into a DataFrame

Application example: compute mean personal income.

## The Process $-$ an Overview

How do we *split* observations by x and *apply* the calculation mean of y?*

<center><img src='https://raw.githubusercontent.com/abjer/sds2017/master/slides/figures/split-apply-combine.png'></center>

## Split-apply-combine in Pandas

A powerful tool in DataFrames is the `groupby` method. Example:

In [70]:
tips.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [33]:
split_var = 'sex' # like x in figure
apply_var = 'total_bill' # like y in figure

tips.groupby(split_var)[apply_var].mean()

sex
Male      20.744076
Female    18.056897
Name: total_bill, dtype: float64

## What is groupby?
`groupby` creats a collection of sub-dataframes we can process.

We can iterate over a groupby object. Example:

In [65]:
results = {}

for group, group_df in tips.groupby('sex'):
      results[group] = group_df.total_bill.mean() 

results

{'Male': 20.744076433121034, 'Female': 18.056896551724137}

In [66]:
pd.Series(results)

Male      20.744076
Female    18.056897
dtype: float64

## Other Methods and Functions in groupby

Can other methods be applied?

- Yes: `mean`, `std`, `min`, `max` all work. 
- Using `.apply()` method and inserting your ***homemade*** function works too.

## Additional variables groupby
Does `groupby` work for multiple variables, functions?

In [67]:
split_vars = ['sex', 'time'] 
apply_vars = ['total_bill', 'tip']
apply_fcts = ['median', 'mean', 'std']
combined = tips.groupby(split_vars)[apply_vars].agg(apply_fcts)

print(combined.reset_index())   

      sex    time total_bill                         tip                    
                      median       mean       std median      mean       std
0    Male   Lunch      16.58  18.048485  7.953435   2.31  2.882121  1.329017
1    Male  Dinner      19.63  21.461452  9.460974   3.00  3.144839  1.529116
2  Female   Lunch      13.42  16.339143  7.500803   2.01  2.582857  1.075108
3  Female  Dinner      17.19  19.213077  8.202085   3.00  3.002115  1.193483


Note grouping with multiple variables uses a [MultiIndex](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.MultiIndex.html) which we do not cover.

## Maintaining the Structure
*How do we get our `groupby` output into the original dataframe?*

- Option 1: you merge it (be careful!)

- Option 2: you use `transform`.

In [36]:
tips['mu_sex'] = tips.groupby(split_vars)[apply_var].transform('mean')
tips.head(5)

,total_bill,tip,sex,smoker,day,time,size,mu_sex
0,16.99,1.01,Female,No,Sun,Dinner,2,19.213077
1,10.34,1.66,Male,No,Sun,Dinner,3,21.461452
2,21.01,3.50,Male,No,Sun,Dinner,3,21.461452
3,23.68,3.31,Male,No,Sun,Dinner,2,21.461452
4,24.59,3.61,Female,No,Sun,Dinner,4,19.213077


In [37]:
tips['dev_sex'] = (tips.total_bill - tips.mu_sex)
tips.head(5)

,total_bill,tip,sex,smoker,day,time,size,mu_sex,dev_sex
0,16.99,1.01,Female,No,Sun,Dinner,2,19.213077,-2.223077
1,10.34,1.66,Male,No,Sun,Dinner,3,21.461452,-11.121452
2,21.01,3.50,Male,No,Sun,Dinner,3,21.461452,-0.451452
3,23.68,3.31,Male,No,Sun,Dinner,2,21.461452,2.218548
4,24.59,3.61,Female,No,Sun,Dinner,4,19.213077,5.376923


# Reshaping Data

## Stacking Data (1:2)

A DataFrame can be collapsed into a Series with the **stack** command.

Let's generate a simple example:

In [71]:
df = pd.DataFrame([[1,2],[3,4]],columns=['EU','US'],index=[2000,2010])
df

,EU,US
2000,1,2
2010,3,4


Now, stack the data:

In [74]:
stacked = df.stack() # going from wide to long format 
stacked

2000  EU    1
      US    2
2010  EU    3
      US    4
dtype: int64

## Stacking Data (2:2)

Clearly, the data now comes with **hierarchical indexing**; first year, then region.

If we want, we can just reset the indexing:

In [75]:
stacked = stacked.reset_index()
stacked

,level_0,level_1,0
0,2000,EU,1
1,2000,US,2
2,2010,EU,3
3,2010,US,4


And then do the renaming...

In [76]:
stacked.columns = ['year', 'place', 'some_val']
stacked

,year,place,some_val
0,2000,EU,1
1,2000,US,2
2,2010,EU,3
3,2010,US,4


## To Wide Format

We can easily transform a *long* DataFrame to *wide* with `unstack()`. Consider the example:

In [77]:
stacked_new = df.stack() # going from wide to long format 
stacked_new

2000  EU    1
      US    2
2010  EU    3
      US    4
dtype: int64

And transform to wide using either the first or the second index:

In [78]:
stacked_new.unstack(level=0)

,2000,2010
EU,1,3
US,2,4


In [79]:
stacked_new.unstack(level=1)

,EU,US
2000,1,2
2010,3,4


## Associated Readings

\[PDA, chapter 7: Revisited... \]

PDA, chapter 8:
- Hierarchical indexing
- Combining and merging data
- Reshaping your data

PDA, chapter 10:
- Introducing `groupby` for data aggregation
- Different ways of undertaking data aggregation
- Introducing the *split-apply-combine*-method

Wickham (2011) \[optional\]:
- Another perspective on the *split-apply-combine*-method for data analysis
- Technical, _R_-based.

## session_4_exercises.ipynb
Can be found on github today 16:00. 
- Method 1: sync your cloned repo
- Method 2: download from git repo

`Remember` to create a local copy of the notebook